In [1]:
from itertools import combinations

import dill as pickle
import evaluate
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from datasets import Dataset
from gensim.models.keyedvectors import KeyedVectors
from ipymarkup import show_span_line_markup
from more_itertools import chunked
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from sentence_transformers import InputExample, SentenceTransformer, losses, models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    DebertaV2ForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)

from snomed_graph import *
import graphistry

In [2]:
annotations_df = pd.read_csv("/workspaces/thyroid_knowledge_graph/1stPlace/data/raw/train_annotations.csv").set_index("note_id")
annotations_df.columns

Index(['start', 'end', 'concept_id'], dtype='object')

In [5]:
annotations_df["note_id"]= annotations_df.index
annotations_df

,start,end,concept_id,note_id
note_id,,,,
10060142-DS-9,179,190,91936005.0,10060142-DS-9
10060142-DS-9,228,248,95563007.0,10060142-DS-9
10060142-DS-9,294,322,45595009.0,10060142-DS-9
10060142-DS-9,390,411,95563007.0,10060142-DS-9
10060142-DS-9,425,444,1835003.0,10060142-DS-9
...,...,...,...,...
19926965-DS-14,9216,9227,76948002.0,19926965-DS-14
19926965-DS-14,9257,9261,22253000.0,19926965-DS-14
19926965-DS-14,9298,9302,22253000.0,19926965-DS-14


In [2]:
SG = SnomedGraph.from_rf2("/workspaces/thyroid_knowledge_graph/1stPlace/data/raw/SnomedCT_RF2_PRODUCTION_20240413T134654Z")



0 terms and 0 relationships were found in the release.
Creating Relationships...


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [2]:
from neo4j import GraphDatabase, basic_auth
from pandas import DataFrame
import graphistry

In [3]:
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=basic_auth("neo4j", "password"))

graphistry.register(api=3, personal_key_id="NHX7JL0YPA",personal_key_secret="EBEUBCKIM52PPWT4" )

In [5]:
# Create User DataFrame by querying Neo4j, converting the results into a pandas DataFrame
with driver.session() as session:
    results = session.run("""
    MATCH (u:User) 
    WITH u.user_key AS screen_name, CASE WHEN "Troll" IN labels(u) THEN 5 ELSE 0 END AS troll
    RETURN screen_name, troll""")
    users = DataFrame(results.data())
# show the first 5 rows of the DataFrame
users[:5]

,screen_name,troll
0,None,0
1,None,0
2,None,0
3,None,0
4,None,0


In [2]:
SG = SnomedGraph.from_rf2("SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z_Challenge_Edition")



FileNotFoundError: [Errno 2] No such file or directory: 'SnomedCT_InternationalRF2_PRODUCTION_20230531T120000Z_Challenge_Edition/Snapshot/Terminology/sct2_Relationship_Snapshot_INT_20230531.txt'

In [ ]:
curl http://localhost:11434/api/generate -d '{
  "model": "medllama2",
  "prompt":"Why is the sky blue?"
}'

In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
random_seed = 42  # For reproducibility
max_seq_len = 512  # Maximum sequence length for (BERT-based) encoders
cer_model_id = "microsoft/deberta-v3-large"  # Base model for Clinical Entity Recogniser
kb_embedding_model_id = ("sentence-transformers/all-MiniLM-L6-v2") # base model for concept encoder
use_LoRA = False  # Whether to use a LoRA to fine-tune the CER model

In [4]:
torch.manual_seed(random_seed)
assert torch.cuda.is_available()

In [5]:
label2id = {"O": 0, "B-clinical_entity": 1, "I-clinical_entity": 2}

id2label = {v: k for k, v in label2id.items()}

In [6]:
cer_tokenizer = AutoTokenizer.from_pretrained(
    cer_model_id, model_max_length=max_seq_len
)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [ ]:
cer_model = DebertaV2ForTokenClassification.from_pretrained(
    cer_model_id, num_labels=3, id2label=id2label, label2id=label2id
)

if use_LoRA:
    lora_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="TOKEN_CLS",
    )

    cer_model = get_peft_model(cer_model, lora_config)

    cer_model.print_trainable_parameters()

In [ ]:

# llm_graph = ChatOpenAI(temperature=0)#, model_name="gpt-4-0125-preview"

In [ ]:
# llm_transformer = LLMGraphTransformer(llm=llm_graph)
# text = """
# Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
# She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
# Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
# She was, in 1906, the first woman to become a professor at the University of Paris.
# """
# documents = [Document(page_content=text)]
# graph_documents = llm_transformer.convert_to_graph_documents(documents)
# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")